## Play with Route 53
* Create an ec2 instance 
* Create a hosted zone
* Create an A record
* Print the name servers 
* Change name servers at domain registrar website 

In [1]:
# pip install boto3 
import secret, time, boto3 # see https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/route53.html 

def AWS_Session(region_name): 
    aws_access_key_id     = secret.aws_access_key_id
    aws_secret_access_key = secret.aws_secret_access_key   
    session = boto3.Session(region_name           = region_name          , 
                            aws_access_key_id     = aws_access_key_id    , 
                            aws_secret_access_key = aws_secret_access_key)    
    return session

session = AWS_Session(region_name='us-east-1')    

In [2]:
from operator import itemgetter 
class EC2:
    def __init__(self, region_name): 
        self.client   = session.client('ec2')
        self.resource = session.resource('ec2')
        self.availability_zone = ''
    
    def choose_first_availability_zone(self):
        self.availability_zone = [i['ZoneName'] for i in self.client.describe_availability_zones()['AvailabilityZones']][0]
        
    def create_key_pair(self, KeyName): # ssh key pair
        self.resource.create_key_pair(KeyName=KeyName)
        
    def get_image_id_of_latest_ubuntu_20_04(self):
        ubuntu_20_04_images = self.client.describe_images(    
            Filters=[
                { 'Name': 'architecture', 'Values': [ 'x86_64', ] },
                { 'Name': 'image-type', 'Values': [ 'machine', ] },
                { 'Name': 'state', 'Values': [ 'available', ] }, 
                { 'Name': 'name', 'Values': [ 'ubuntu/images/hvm-ssd/ubuntu-focal-20.04-amd64-server-????????', ] },]
        )['Images']
        return sorted(ubuntu_20_04_images, key=itemgetter('CreationDate'), reverse=True)[0]['ImageId']

ec2 = EC2(region_name='us-east-1')
ec2.choose_first_availability_zone()

In [3]:
class Route53:
    def __init__(self):  
        self.client = session.client('route53')  

route53 = Route53()

### Create an instance for test

In [4]:
KeyName = 'ec2-us-east-1'
ImageId = ec2.get_image_id_of_latest_ubuntu_20_04() 
myInstances = ec2.resource.create_instances(  
    UserData='''#!/bin/bash
uptime > /tmp/uptime
cat << EOF >> /root/hehe.sh
date > /tmp/test
EOF
(crontab -l 2>/dev/null; echo "@reboot date > /tmp/date") | crontab -''', # The UserData is a script executed only once by root at launching
    KeyName=KeyName,              # ssh key
    ImageId=ImageId,  
    InstanceType='t2.micro', # Free tier
    MaxCount=1, MinCount=1,
    EbsOptimized=False,   
    NetworkInterfaces=[
        {
            'AssociatePublicIpAddress': True,
            'DeleteOnTermination': True, 
            'DeviceIndex': 0,   
        }, 
    ] 
)
myInstance = myInstances[0]

myInstance.wait_until_running() 
myInstance.reload() # update public ip
print('Public IP:', myInstance.public_ip_address)  

Public IP: 54.88.58.71


### Create a hosted/DNS zone

#### Domain name yzlab.group bought @ Google Domain

In [5]:

myDomainName = 'yzlab.group'


In [6]:
response = route53.client.create_hosted_zone(
    Name=myDomainName, # Domain name 
    CallerReference='uniqueString',
    HostedZoneConfig={
        'Comment': 'crazySheet',
        'PrivateZone': False
    }
)
myHostedZoneId = response['HostedZone']['Id']

### An A type record

In [7]:
response = route53.client.change_resource_record_sets(
    HostedZoneId=myHostedZoneId, # 
    ChangeBatch= {'Comment': 'add an A type record',
                  'Changes': [{
                       'Action': 'UPSERT',
                       'ResourceRecordSet': {
                           'Name': myDomainName,
                           'ResourceRecords': [{'Value': myInstance.public_ip_address}], # 
                           'Type': 'A',
                           'TTL': 300,
}}]})

### The printed nameservers are for Google Domains DNS tab

In [8]:
paginator = route53.client.get_paginator('list_resource_record_sets')
 
for record_set in paginator.paginate(HostedZoneId=myHostedZoneId): 
    for record in record_set['ResourceRecordSets']: 
        if record['Type'] == 'NS':
            for nameserver in record['ResourceRecords']:
                print(nameserver['Value']) 

ns-97.awsdns-12.com.
ns-897.awsdns-48.net.
ns-1709.awsdns-21.co.uk.
ns-1270.awsdns-30.org.


### Change DNS settings @ [Google](https://support.google.com/domains/answer/3290309) and wait several minites before it takes effect.
![Structure](https://github.com/xg590/tutorials/blob/master/AWS/GoogleDomainDNS.png?raw=true)